In [1]:
import os
import time 

import ase.io 
import numpy as np 
from ase import units
from ase.md.langevin import Langevin
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution

from mace.calculators.mace import MACECalculator 

# Functions

In [23]:
def printenergy(dyn, start_time=None):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    a = dyn.atoms
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    if start_time is None:
        elapsed_time = 0
    else:
        elapsed_time = time.time() - start_time
    
    print(a.calc.results.keys())
    forces_var = np.var(a.calc.results["forces"], axis=0)
    print(
        "%.1fs: Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  "  # pylint: disable=C0209
        "Etot = %.3feV t=%.1ffs Eerr = %.3feV Ferr = %.3feV/A"
        % (
            elapsed_time,
            epot,
            ekin,
            ekin / (1.5 * units.kB),
            epot + ekin,
            dyn.get_time() / units.fs,
            np.var(a.calc.results["energy"]),
            np.linalg.norm(forces_var),
        ),
        flush=True,
    )
def _printenergy(dyn, start_time):
    a = dyn.atoms
    if start_time is None:
        start_time = time.time()
        elapsed_time = 0
    else:
        elapsed_time = time.time() - start_time
    
    print("Available keys:", a.calc.results.keys())
    forces = a.calc.results["forces"]
    print("Shape of forces array:", forces.shape)
    
    # Calculate the variance of forces
    forces_var = np.var(forces, axis=0)
    
    # Calculate energies
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    
    print(
        "%.1fs: Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  "
        "Etot = %.3feV t=%.1ffs Eerr = %.3feV Ferr = %.3feV/A"
        % (
            elapsed_time,
            epot,
            ekin,
            a.get_temperature(),
            epot + ekin,
            dyn.get_time() / units.fs,
            np.var(a.calc.results["energy"]),
            np.linalg.norm(forces_var),  # Remove axis argument
        ),
        flush=True,
    )

def save_config(dyn, fname):
    atomsi = dyn.atoms
    ens = atomsi.get_potential_energy()
    frcs = atomsi.get_forces()

    print(f"Energy on frame : {atomsi.calc.results['energy']} ")
    print(f"Forces on frame with dim = {np.array(atomsi.calc.results['forces']).shape} : {atomsi.calc.results['forces']} ")

    atomsi.info.update(
        {
            "mlff_energy": ens,
            "time": np.round(dyn.get_time() / units.fs, 5),
            "mlff_energy_var": np.var(atomsi.calc.results["energy"]),
        }
    )
    atomsi.arrays.update(
        {
            "mlff_forces": frcs,
            "mlff_forces_var": np.var(atomsi.calc.results["forces"], axis=0),
        }
    )

    ase.io.write(fname, atomsi, append=True)
    
def _save_config(dyn, fname):
    atomsi = dyn.atoms
    ens = atomsi.get_potential_energy()
    frcs = atomsi.get_forces()

    atomsi.info.update(
        {
            "mlff_energy": ens,
            "time": np.round(dyn.get_time() / units.fs, 5),
            "mlff_energy_var": np.var(atomsi.calc.results["energy"]),
        }
    )
    
    # Ensure forces_var has the correct shape
    forces_var = np.var(atomsi.calc.results["forces"], axis=0)
    if forces_var.shape != frcs.shape:
        forces_var = np.tile(forces_var, (len(atomsi), 1))
    
    atomsi.arrays.update(
        {
            "mlff_forces": frcs,
            "mlff_forces_var": forces_var,
        }
    )

    # Debug print
    print("Shape of mlff_forces:", frcs.shape)
    print("Shape of mlff_forces_var:", forces_var.shape)

    ase.io.write(fname, atomsi, append=True)

def _save_config(dyn, fname):
    atomsi = dyn.atoms
    ens = atomsi.get_potential_energy()
    frcs = atomsi.get_forces()

    print("All keys in calc.results:", atomsi.calc.results.keys())

    # Handle committee results
    energy_committee = atomsi.calc.results.get("energies", [])
    forces_committee = atomsi.calc.results.get("forces_comm", [])

    atomsi.info.update({
        "mlff_energy": ens,
        "time": np.round(dyn.get_time() / units.fs, 5),
        "mlff_energy_var": np.var(energy_committee),
        "mlff_energy_committee": energy_committee,
    })

    atomsi.arrays.update({
        "mlff_forces": frcs,
        "mlff_forces_var": np.var(forces_committee, axis=0),
        "mlff_forces_committee": forces_committee,
    })

    # Handle other results
    for key, value in atomsi.calc.results.items():
        if key not in ["energies", "forces_comm", "energy", "forces"]:
            if np.array(value).shape == (8,):
                atomsi.info[f"mlff_{key}"] = value
            elif np.array(value).shape == (124, 3):
                atomsi.arrays[f"mlff_{key}"] = value
            else:
                print(f"Skipping {key} with shape {np.array(value).shape}")

    # Debug print
    print("Shapes in atomsi.info:")
    for key, value in atomsi.info.items():
        print(f"  {key}: {np.array(value).shape}")
    print("Shapes in atomsi.arrays:")
    for key, value in atomsi.arrays.items():
        print(f"  {key}: {np.array(value).shape}")

    try:
        ase.io.write(fname, atomsi, append=True)
    except ValueError as e:
        print(f"Error writing to file: {e}")
        print("atomsi info:", atomsi.info)
        print("atomsi arrays keys:", atomsi.arrays.keys())
        raise
    
def stop_error(dyn, threshold, reg=0.2):
    atomsi = dyn.atoms
    force_var = np.var(atomsi.get_forces(), axis=0)
    force = atomsi.get_forces()
    ferr = np.sqrt(np.sum(force_var, axis=1))
    ferr_rel = ferr / (np.linalg.norm(force, axis=1) + reg)

    if np.max(ferr_rel) > threshold:
        print(
            "Error too large {:.3}. Stopping t={:.2} fs.".format(  # pylint: disable=C0209
                np.max(ferr_rel), dyn.get_time() / units.fs
            ),
            flush=True,
        )
        dyn.max_steps = 0

def run(model, atoms_fname, atoms_index, output, device='cpu', default_dtype='float64', nsave=10, nsteps=1000, timestep=1,temperature_K=300,friction=0.01,ncheckerror=10,error_threshold=0.1) -> None:
    """
    Run molecular dynamics simulation using the MACE calculator.

    Parameters:
        model (str): Path to the MACE model file.
        atoms_fname (str): Path to the input file containing atomic structure.
        atoms_index (int): Index of the atomic structure to read from the input file.
        output (str): Path to the output file where the trajectory will be saved.
        device (str, optional): Device to run the simulation on ('cpu' or 'gpu'). Default is 'cpu'.
        default_dtype (str, optional): Default data type for calculations. Default is 'float64'.
        nsave (int, optional): Number of steps between saving configurations. Default is 10.
        nsteps (int, optional): Total number of steps to run the simulation. Default is 1000.
        timestep (float, optional): Time step for the simulation in femtoseconds. Default is 1.
        temperature_K (float, optional): Temperature in Kelvin for the simulation. Default is 300.
        friction (float, optional): Friction coefficient for the Langevin dynamics. Default is 0.01.
        ncheckerror (int, optional): Interval for checking errors during the simulation. Default is 10.
        error_threshold (float, optional): Threshold for stopping the simulation if errors exceed this value. Default is 0.1.

    Returns:
        None
    """
    mace_fname = model
    atoms_fname = atoms_fname
    atoms_index = atoms_index

    mace_calc = MACECalculator(
        mace_fname,
        device,
        default_dtype=default_dtype,
    )

    NSTEPS = nsteps

    if os.path.exists(output) and os.path.getsize(output) > 0:
        print("Trajectory exists. Continuing from last step.")
        atoms = ase.io.read(output, index=-1)
        len_save = len(ase.io.read(output, ":"))
        print("Last step: ", atoms.info["time"], "Number of configs: ", len_save)
        NSTEPS -= len_save * nsave
    else:
        atoms = ase.io.read(atoms_fname, index=atoms_index)
        MaxwellBoltzmannDistribution(atoms, temperature_K=temperature_K)
    #print(atoms)
    atoms.calc = mace_calc
    print(atoms.calc.num_models)
    print(atoms.calc.models)
    #print(dir(mace_calc))

    # We want to run MD with constant energy using the Langevin algorithm
    # with a time step of 5 fs, the temperature T and the friction
    # coefficient to 0.02 atomic units.
    dyn = Langevin(
        atoms=atoms,
        timestep=timestep * units.fs,
        temperature_K=temperature_K,
        friction=friction,
    )

    dyn.attach(printenergy, interval=nsave, dyn=dyn, start_time=time.time())
    dyn.attach(save_config, interval=nsave, dyn=dyn, fname=output)
    dyn.attach(
        stop_error, interval=ncheckerror, dyn=dyn, threshold=error_threshold
    )
    # Now run the dynamics
    dyn.run(NSTEPS)




# Run the Code

In [ ]:
#mace_model_path = '../Potentials/vcrtiwzr_fep_vac_r6_e1_f25_s100_L2_all_correct_stress_stagetwo_compiled.model'
mace_model_path = './finished_runs/2024-09-27_vcrtiwzr_fep+vac+neb+perf/models_out/vcrtiwzr_all_e1_f50_25s_seed_*_compiled.model'
atoms_fname = './data/fep_vac_neb_perf_test.xyz'
atoms_index = 0

run_num = 1
output_dir = f'./Mace_Active_Learning/MD_Run_{run_num}'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
output = os.path.join(output_dir, 'trajectory.extxyz')

device = 'cpu'
nsteps = 100
timestep = 5
temperature_K = 300
friction=0.01

run(model=mace_model_path,
    atoms_fname=atoms_fname,
    atoms_index=atoms_index,
    output=output,
    device=device,
    nsteps=nsteps,
    timestep=timestep,
    temperature_K=temperature_K,
    friction=friction)

In [1]:
from ase.io import read
from mace.calculators.mace import MACECalculator
mace_model_path = './finished_runs/2024-09-27_vcrtiwzr_fep+vac+neb+perf/models_out/vcrtiwzr_all_e1_f50_25s_seed_*_compiled.model'
atoms_fname = './data/fep_vac_neb_perf_test.xyz'

mace_calc = MACECalculator(mace_model_path, device='cpu', default_dtype='float64')
atoms = read(atoms_fname)

atoms.calc = mace_calc

print(atoms.get_potential_energy())


Running committee mace with 8 models


/opt/homebrew/Caskroom/miniforge/base/envs/allegro/lib/python3.10/site-packages/torch/serialization.py:779: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn("'torch.load' received a zip file that looks like a TorchScript archive"


-1163.6215307458183


In [27]:
import os 
import numpy as np 
atoms_list = read(atoms_fname,index=':')
atoms_list = atoms_list[0:5]

mace_model_path = 'finished_runs/2024-09-27_vcrtiwzr_fep+vac+neb+perf/models_out'
mace_models = [file for file in os.listdir(mace_model_path) if file.endswith('.model')]

comms_data = []
for i,atoms in enumerate(atoms_list):
    print(f"On atom {i+1}/{len(atoms_list)}")
    comm_results = {
        'energies' : [],
        'forces' : [],
        'stresses' : []
    }
    for model in mace_models:
        mace_calc = MACECalculator(os.path.join(mace_model_path,model), device='cpu', default_dtype='float64')
        atoms.calc = mace_calc
        comm_results['energies'].append(atoms.get_potential_energy())
        comm_results['forces'].append(atoms.get_forces())
        comm_results['stresses'].append(atoms.get_stress())

    comm_results['e_var'] = np.var(comm_results['energies'])
    comm_results['f_var'] = np.var(comm_results['forces'], axis=0)
    comm_results['s_var'] = np.var(comm_results['stresses'], axis=0)
    #atoms.info['comm_results'] = comm_results
    comm_results['atoms'] = atoms.todict()
    comms_data.append(comm_results)


On atom 1/5


/opt/homebrew/Caskroom/miniforge/base/envs/allegro/lib/python3.10/site-packages/torch/serialization.py:779: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn("'torch.load' received a zip file that looks like a TorchScript archive"


On atom 2/5
On atom 3/5
On atom 4/5
On atom 5/5


In [27]:
from ase import Atoms
print(comms_data[0]['atoms']['numbers'])

test_atoms = Atoms(numbers=comms_data[0]['atoms']['numbers'], 
                   positions=comms_data[0]['atoms']['positions'], 
                   cell=comms_data[0]['atoms']['cell'], 
                   pbc=comms_data[0]['atoms']['pbc'],
                   info=comms_data[0]['atoms']['info'])

[40 40 40 40 40 40 40 40 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22 23
 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23
 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23
 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23
 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 23 24 24 24 24
 24 74 74 74]


In [18]:
print(atoms_list[1].info['comm_results']['e_var'])
print(atoms_list[1].info['comm_results']['f_var'])
print(atoms_list[1].info['comm_results']['s_var'])

24.399820877046324
[[1.52327605e-03 2.76612393e-03 1.35546042e-03]
 [1.01082531e-03 8.26947994e-04 1.61945772e-03]
 [2.19746512e-04 5.09098343e-04 7.05752264e-04]
 [5.34019214e-04 1.78503363e-03 9.36202201e-04]
 [2.66416545e-04 2.33432748e-03 7.45446864e-04]
 [1.02614521e-03 6.78977183e-04 3.94163188e-03]
 [1.02567947e-03 2.26927194e-03 1.24519564e-03]
 [7.94527084e-04 8.75216245e-04 7.08948954e-04]
 [1.43714667e-03 4.82473077e-04 1.72559893e-03]
 [7.84617059e-04 2.01493872e-03 1.89374981e-03]
 [5.77198400e-04 5.62933410e-04 4.18722871e-04]
 [8.56107921e-04 1.25505142e-03 2.07412736e-03]
 [1.19372807e-03 5.33435896e-03 1.21658401e-03]
 [8.74569721e-05 2.47646029e-04 3.21429305e-03]
 [2.39639996e-03 2.08840110e-03 2.66352566e-03]
 [3.17757620e-03 4.68649724e-03 6.82964209e-04]
 [3.75691124e-03 7.86985423e-03 7.66658769e-04]
 [2.08486522e-03 6.12886303e-04 3.82898004e-03]
 [1.71644570e-03 5.99157367e-03 9.68373188e-04]
 [6.36362044e-03 2.42718901e-03 2.50003078e-03]
 [2.19409476e-03 7.80

In [28]:
from ase.io import write
import pickle as pkl
with open('comm_results.pkl','wb') as f:
    pkl.dump(comms_data,f)

In [33]:
def dict_to_atoms(atoms_dict):
    atoms = Atoms(numbers=atoms_dict['numbers'], 
                  positions=atoms_dict['positions'], 
                  cell=atoms_dict['cell'], 
                  pbc=atoms_dict['pbc'],
                  info=atoms_dict['info'])
    return atoms

with open('comm_results.pkl','rb') as f:
    data = pkl.load(f)

entries = data 
sorted_entries = sorted(entries, key=lambda x: np.sum(x['f_var']), reverse=True)
top_25_entries = sorted_entries[:25]
atoms_list = [dict_to_atoms(entry['atoms']) for entry in top_25_entries]

In [5]:
import numpy as np
from ase import Atoms
from scipy.spatial.distance import pdist, squareform
import warnings
from ase.io import read
from mace.calculators.mace import MACECalculator
import os

def maximize_force_variance(atoms, ensemble_calculators, temperature=300, max_iterations=1000, min_distance=2.0):
    """
    Maximize the normalized variance of forces on atoms using an ensemble of calculators,
    with temperature-based displacements and detailed debugging output.
    
    Args:
    atoms (ase.Atoms): The atomic configuration to optimize
    ensemble_calculators (list): List of ASE-compatible calculators
    temperature (float): Temperature for atomic displacements (in Kelvin)
    max_iterations (int): Maximum number of optimization iterations
    min_distance (float): Minimum allowed distance between atoms (in Angstroms)
    
    Returns:
    ase.Atoms: Optimized atomic configuration
    np.array: Per-atom normalized force variances
    float: Mean free path (average distance traveled by atoms)
    """
    
    kB = 8.617333262e-5  # Boltzmann constant in eV/K
    
    def calculate_normalized_force_variances(positions):
        try:
            atoms.set_positions(positions.reshape(-1, 3))
            all_forces = np.array([calc.get_forces(atoms) for calc in ensemble_calculators])
            
            print(f"Debug: Shape of all_forces: {all_forces.shape}")
            print(f"Debug: Range of forces: {np.min(all_forces)} to {np.max(all_forces)}")
            
            forces_magnitude = np.linalg.norm(all_forces, axis=2)
            print(f"Debug: Range of force magnitudes: {np.min(forces_magnitude)} to {np.max(forces_magnitude)}")
            
            with np.errstate(divide='ignore', invalid='ignore'):
                normalized_forces = np.where(forces_magnitude[:, :, np.newaxis] != 0,
                                             all_forces / forces_magnitude[:, :, np.newaxis],
                                             0)
            
            print(f"Debug: Range of normalized forces: {np.min(normalized_forces)} to {np.max(normalized_forces)}")
            
            variances = np.var(normalized_forces, axis=0)
            print(f"Debug: Range of variances: {np.min(variances)} to {np.max(variances)}")
            
            if np.any(np.isnan(variances)) or np.any(np.isinf(variances)):
                print("Warning: NaN or Inf in force variances")
                return None
            return variances
        except Exception as e:
            print(f"Error in force calculation: {str(e)}")
            return None
    
    def check_min_distance(positions):
        dist_matrix = squareform(pdist(positions.reshape(-1, 3)))
        np.fill_diagonal(dist_matrix, np.inf)
        min_dist = np.min(dist_matrix)
        print(f"Debug: Minimum interatomic distance: {min_dist}")
        return min_dist >= min_distance
    
    def objective_function(positions):
        if not check_min_distance(positions):
            print("Debug: Minimum distance constraint violated")
            return -np.inf
        force_variances = calculate_normalized_force_variances(positions)
        if force_variances is None:
            print("Debug: Force variances calculation failed")
            return -np.inf
        total_variance = np.sum(force_variances)
        print(f"Debug: Total variance: {total_variance}")
        return total_variance
    
    initial_positions = atoms.get_positions().flatten()
    current_positions = initial_positions.copy()
    current_variance = objective_function(current_positions)
    
    if np.isinf(current_variance) or np.isnan(current_variance):
        print("Error: Initial configuration results in invalid variance.")
        return atoms, None, 0.0
    
    total_distances = np.zeros(len(atoms))
    
    for step in range(max_iterations):
        displacement = np.random.normal(0, np.sqrt(kB * temperature), current_positions.shape)
        print(f"Debug: Step {step}, Max displacement: {np.max(np.abs(displacement))}")
        
        proposed_positions = current_positions + displacement
        
        if not check_min_distance(proposed_positions):
            print("Debug: Proposed positions violated minimum distance")
            continue
        
        new_variance = objective_function(proposed_positions)
        
        if np.isinf(new_variance) or np.isnan(new_variance):
            print("Debug: New variance is invalid")
            continue
        
        acceptance_probability = np.exp((new_variance - current_variance) / (kB * temperature))
        print(f"Debug: Acceptance probability: {acceptance_probability}")
        
        if new_variance > current_variance or np.random.random() < acceptance_probability:
            distances = np.linalg.norm(proposed_positions.reshape(-1, 3) - current_positions.reshape(-1, 3), axis=1)
            total_distances += distances
            
            current_positions = proposed_positions
            current_variance = new_variance
            print("Debug: Move accepted")
        else:
            print("Debug: Move rejected")
        
        print(f"Step {step}: Total normalized force variance = {current_variance:.6f}")
    
    atoms.set_positions(current_positions.reshape(-1, 3))
    final_variances = calculate_normalized_force_variances(current_positions)
    
    mean_free_path = np.mean(total_distances)
    
    print(f"\nInitial total normalized force variance: {objective_function(initial_positions):.6f}")
    print(f"Final total normalized force variance: {current_variance:.6f}")
    print(f"Change in total normalized force variance: {current_variance - objective_function(initial_positions):.6f}")
    print(f"Mean free path (average distance traveled by atoms): {mean_free_path:.6f} Angstroms")
    
    return atoms, final_variances, mean_free_path


In [21]:
import numpy as np
from ase import Atoms
from scipy.spatial.distance import pdist, squareform

def maximize_force_variance(atoms, ensemble_calculators, initial_temperature=5000, final_temperature=300, max_iterations=1000, min_distance=1.0):
    kB = 8.617333262e-5  # Boltzmann constant in eV/K
    
    def calculate_normalized_force_variances(positions):
        try:
            atoms.set_positions(positions.reshape(-1, 3))
            all_forces = np.array([calc.get_forces(atoms) for calc in ensemble_calculators])
            forces_magnitude = np.linalg.norm(all_forces, axis=2)
            with np.errstate(divide='ignore', invalid='ignore'):
                normalized_forces = np.where(forces_magnitude[:, :, np.newaxis] != 0,
                                             all_forces / forces_magnitude[:, :, np.newaxis],
                                             0)
            variances = np.var(normalized_forces, axis=0)
            return variances
        except Exception as e:
            print(f"Error in force calculation: {str(e)}")
            return None
    
    def check_min_distance(positions):
        dist_matrix = squareform(pdist(positions.reshape(-1, 3)))
        np.fill_diagonal(dist_matrix, np.inf)
        return np.min(dist_matrix) >= min_distance
    
    def objective_function(positions):
        if not check_min_distance(positions):
            return -np.inf
        force_variances = calculate_normalized_force_variances(positions)
        if force_variances is None:
            return -np.inf
        return np.sum(force_variances)
    
    current_positions = atoms.get_positions().flatten()
    current_variance = objective_function(current_positions)
    
    if np.isinf(current_variance) or np.isnan(current_variance):
        print("Error: Initial configuration results in invalid variance.")
        return atoms, None, 0.0
    
    total_distances = np.zeros(len(atoms))
    temperatures = np.linspace(initial_temperature, final_temperature, max_iterations)
    
    variance_history = [current_variance]
    accepted_moves = 0
    
    for step in range(max_iterations):
        current_temperature = temperatures[step]
        displacement_scale = np.sqrt(kB * current_temperature)
        
        displacement = np.random.normal(0, displacement_scale, current_positions.shape)
        proposed_positions = current_positions + displacement
        
        if not check_min_distance(proposed_positions):
            continue
        
        new_variance = objective_function(proposed_positions)
        
        if np.isinf(new_variance) or np.isnan(new_variance):
            continue
        
        delta_variance = new_variance - current_variance
        variance_std = np.std(variance_history) if len(variance_history) > 1 else 1.0
        
        acceptance_probability = min(1, np.exp(delta_variance / (variance_std * np.sqrt(step + 1))))
        
        if delta_variance > 0 or np.random.random() < acceptance_probability:
            current_positions = proposed_positions
            current_variance = new_variance
            variance_history.append(current_variance)
            accepted_moves += 1
            
            distances = np.linalg.norm(displacement.reshape(-1, 3), axis=1)
            total_distances += distances
            
            print(f"Step {step}: Move accepted. New variance: {current_variance:.6f}")
        else:
            print(f"Step {step}: Move rejected. Current variance: {current_variance:.6f}")
        
        print(f"Step {step}: Total normalized force variance = {current_variance:.6f}, Acceptance rate: {accepted_moves/(step+1):.4f}")
    
    final_atoms = atoms.copy()
    final_atoms.set_positions(current_positions.reshape(-1, 3))
    final_variances = calculate_normalized_force_variances(current_positions)
    
    mean_free_path = np.mean(total_distances)
    
    print(f"\nInitial total normalized force variance: {objective_function(atoms.get_positions().flatten()):.6f}")
    print(f"Final total normalized force variance: {current_variance:.6f}")
    print(f"Total accepted moves: {accepted_moves}/{max_iterations}")
    print(f"Final acceptance rate: {accepted_moves/max_iterations:.4f}")
    print(f"Mean free path (average distance traveled by atoms): {mean_free_path:.6f} Angstroms")
    
    return final_atoms, final_variances, mean_free_path

# Example usage remains the same

In [22]:
atoms_fname = './data/fep_vac_neb_perf_test.xyz'
atoms_list = read(atoms_fname,index=':')
atoms_list = atoms_list[0:2]

# Example usage:
mace_model_path = 'finished_runs/2024-09-27_vcrtiwzr_fep+vac+neb+perf/models_out'
mace_models = [file for file in os.listdir(mace_model_path) if file.endswith('.model')]
ensemble_calculators = [MACECalculator(os.path.join(mace_model_path,model), device='cpu', default_dtype='float64') for model in mace_models]

optimized_atoms = []
atoms = atoms_list[1].copy()
steps = 1000
steps_per_run = 1000
temperature = 5000

for i in range(int(steps/steps_per_run)):
    new_atoms, final_variances, mean_free_path = maximize_force_variance(
        atoms=atoms,
        ensemble_calculators=ensemble_calculators, 
        initial_temperature=temperature,
        final_temperature=300, 
        max_iterations=steps_per_run, 
        min_distance=1.25
    )
    optimized_atoms.append(new_atoms)
    atoms = new_atoms  # Use the new atoms object for the next iteration
    print(f"Step {i+1}/{steps/steps_per_run} completed.")
    
    # Print some statistics to verify changes
    initial_positions = atoms_list[1].get_positions()
    final_positions = new_atoms.get_positions()
    max_displacement = np.max(np.linalg.norm(final_positions - initial_positions, axis=1))
    print(f"Maximum atomic displacement: {max_displacement:.6f} Angstroms")

Step 3: Move rejected. Current variance: 61.175518
Step 3: Total normalized force variance = 61.175518, Acceptance rate: 0.0000
Step 16: Move rejected. Current variance: 61.175518
Step 16: Total normalized force variance = 61.175518, Acceptance rate: 0.0000
Step 26: Move accepted. New variance: 62.564332
Step 26: Total normalized force variance = 62.564332, Acceptance rate: 0.0370
Step 325: Move rejected. Current variance: 62.564332
Step 325: Total normalized force variance = 62.564332, Acceptance rate: 0.0031
Step 422: Move accepted. New variance: 56.357625
Step 422: Total normalized force variance = 56.357625, Acceptance rate: 0.0047
Step 495: Move accepted. New variance: 56.555038
Step 495: Total normalized force variance = 56.555038, Acceptance rate: 0.0060
Step 754: Move accepted. New variance: 55.547676
Step 754: Total normalized force variance = 55.547676, Acceptance rate: 0.0053
Step 814: Move accepted. New variance: 63.227064
Step 814: Total normalized force variance = 63.2270

In [23]:
from ase.io import write
out_path = 'Mace_Active_Learning/Force_Variance_Optimization_T4'
if not os.path.exists(out_path):
    os.makedirs(out_path)
# write all the atoms objects 
for i,atoms in enumerate(optimized_atoms):
    write(os.path.join(out_path,f'optimized_atoms_{i}.xyz'),atoms)